<a href="https://colab.research.google.com/github/Iispar/dl-in-hlt-project/blob/main/course_project_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep learning in Human Language Technology Project (Template)

- Student(s) Name(s): Iiro Partanen
- Date: 18/10/2023
- Chosen Corpus: amazon_reviews_multi
- Contributions (if group project): -

### Corpus information

- Description of the chosen corpus:
- Paper(s) and other published materials related to the corpus:
- Random baseline performance and expected performance for recent machine learned models:

---

## 1. Setup

In [1]:
!pip3 install -q transformers datasets evaluate
!pip install optuna
import datasets
import sklearn.feature_extraction
import torch
import transformers
import numpy as np
import evaluate
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 129.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

---

## 2. Data download, sampling and preprocessing

### 2.1. Download the corpus

In [21]:
engDataset = datasets.load_dataset('mteb/amazon_reviews_multi', name='en'); # imports the dataset.
# check it works
print(engDataset);

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})


### 2.2. Sampling and preprocessing

In [22]:
engDataset = engDataset.shuffle() # shuffle the dataset for safety.
engDataset = engDataset.remove_columns(['id', 'label_text']) # removes everything that we don't need

{'text': 'Kept my interest\n\nTerrific read. Very scary.', 'label': 4}


In [26]:
# lets look at five results to see if there is more preprocessing to be done

print(engDataset['train'][:5])

# looks like the title is spaced with \n\n, but other than that there is no problems. Looks good to me.

{'text': ['Kept my interest\n\nTerrific read. Very scary.', 'worth the time\n\nIsland of Grace is heartwarming and inspiring. We need to take our circumstances to Jesus, who wants to love and care for you, they can never be too much for God. The acting was well done and it is a clean movie-what a relief!', 'Do not buy- very thin\n\nThese adhesive bra are so thin that there is no difference putting it on. I have been using adhesive bras for a while now and this is the worst purchase I had. waste of money.', 'Key Pad useless\n\nNum lock key will not work. I have been all over the internet and there is no information about how to get the key to work. I have used every control and function key in combination. My number pad is useless. And both num lock and caps lock lights do not work.', 'A little disappointed\n\nNo screen protector. Not sure this would take the 3 feet drop test on hard floor. Light weight with good grip. Expecting more.'], 'label': [4, 3, 0, 0, 2]}


---

## 3. Machine learning model

### 3.1. Model training

In [ ]:
# Your code to train the transformer based model on the training set and evaluate the performance on the validation set here

### 3.2 Hyperparameter optimization

In [ ]:
# Your code for hyperparameter optimization here

### 3.3. Evaluation on test set

In [ ]:
# Your code to evaluate the final model on the test set here

### 3.4. Multilingual and cross-lingual experiments

In [ ]:
# Your code to train and evaluate the multilingual and cross-lingual models

---

## 4. Results and summary

### 4.1 Corpus insights

(Briefly discuss what you learned about the corpus and its annotation)

### 4.2 Results

(Briefly summarize your results)

### 4.3 Relation to random baseline / expected performance / state of the art

(Compare your results to the random and state-of-the-art performance)

---

## 5. Bonus Task (optional)

### 5.1. Data selection

(Briefly describe how many English and target language examples were used and how these were selected, include relevant code)

### 5.2 Sentence representations

In [ ]:
# Your code to create a sentence embedding for the given text here

### 5.3. Cosine similarity

In [ ]:
# Your code to calculate the cosine similarity of the embeddings and select the target sentence that maximizes the cosine similarity here

### 5.4 Bonus task evaluation

(Present the evaluation results here)